<a href="https://colab.research.google.com/github/k-rajmani2k/Generative-AI_Chatbot/blob/main/Gen_Ai_part_p3_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# Load the text file and preprocess the text
filename = "wonderland.txt"
with open(filename, "r", encoding="utf-8") as file:
    raw_text = file.read()

raw_text = raw_text.lower()  # Convert to lowercase to normalize text
print("First 500 characters of text:")
print(raw_text[:500])  # Display the first 500 characters of the text

# Create a sorted list of unique characters in the text
chars = sorted(list(set(raw_text)))

print(f"Unique characters: {chars}")
print(f"Number of unique characters: {len(chars)}")


char_to_int = dict((c,i) for i,c in enumerate(chars))


# # Create mappings for characters to integers and vice versa
# char_to_int = {char: idx for idx, char in enumerate(chars)}
# int_to_char = {idx: char for idx, char in enumerate(chars)}

# print("Character-to-Integer mapping example:")
# print({key: char_to_int[key] for key in list(char_to_int)[:10]})  # Show a sample mapping


First 500 characters of text:
alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “without pictures or conversations?” so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getti
Unique characters: ['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '’', '“', '”']
Number of unique characters: 41


In [2]:
n_char = len(raw_text)
n_vocab = len(chars)
print("Total characters:", n_char)
print("Total unique characters:", n_vocab)

Total characters: 13637
Total unique characters: 41


In [3]:
seq_length = 100
dataX = []
dataY = []

# Loop to create input-output pairs
for i in range(0, len(raw_text) - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]  # Sequence of 100 characters
    seq_out = raw_text[i + seq_length]  # Next character (the target)
    dataX.append([char_to_int[char] for char in seq_in])  # Convert input characters to integers
    dataY.append(char_to_int[seq_out])  # Convert output character to integer

n_patterns = len(dataX)
print("Total patterns:", n_patterns)


Total patterns: 13537


In [4]:
len(dataX)

13537

In [5]:
len(dataY)

13537

In [6]:
# Reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length,1))
X = X / float(n_vocab)
# One-hot encode the output variable
y = to_categorical(dataY, num_classes=len(chars))  # Specify the number of classes explicitly

print("Input shape (X):", X.shape)  # Expected: (n_patterns, seq_length, 1)
print("Output shape (y):", y.shape)  # Expected: (n_patterns, n_vocab)




Input shape (X): (13537, 100, 1)
Output shape (y): (13537, 41)


In [7]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1], X.shape[2])))  # Corrected input shape
model.add(Dropout(0.2))  # Dropout to reduce overfitting
model.add(Dense(len(chars), activation="softmax"))  # Output layer with softmax activation

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 256)                 │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 41)                  │          10,537 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 274,729 (1.05 MB)

 Trainable params: 274,729 (1.05 MB)

 Non-trainable params: 0 (0.00 B)

None


In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the file path for saving the model weights, ensuring it ends with '.keras'
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.keras" # Changed '.hdf5' to '.keras'

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True)

# List of callbacks
callbacks_list = [checkpoint]

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam")

# Fit the model
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)


Epoch 1/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.1429
Epoch 1: loss improved from inf to 3.01912, saving model to weights-improvement-01-3.0191.keras
106/106 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 3.1418
Epoch 2/50
104/106 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.9581
Epoch 2: loss improved from 3.01912 to 2.95788, saving model to weights-improvement-02-2.9579.keras
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 2.9581
Epoch 3/50
103/106 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.9565
Epoch 3: loss improved from 2.95788 to 2.95145, saving model to weights-improvement-03-2.9515.keras
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 2.9563
Epoch 4/50
105/106 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.9481
Epoch 4: loss improved from 2.95145 to 2.94504, saving model to weights-improvement-04-2.9450.keras
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2.9480
Epoch 5/50
105/106 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.9423
Epoch 5: loss improved from 2.94504 

In [ ]:
# # # Fit the model
# model.fit(X, y, epochs=4, batch_size=128, callbacks=callbacks_list)


In [9]:
filename = "/content/weights-improvement-50-1.2708.keras"

In [10]:
model.load_weights(filename)
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [11]:
# With this line:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
import sys
import numpy

In [ ]:
import numpy as np

# Generate a random starting index
start = np.random.randint(0, len(dataX))  # Random index
pattern = dataX[start]

# Print the seed
print("Seed:")
print("'{}'".format(''.join([int_to_char[value] for value in pattern])))

# Generate characters
for i in range(1000):
    # Reshape the input pattern for the model
    x = np.reshape(pattern, (1, len(pattern), 1))
    # Normalize the values
    x = x / float(n_vocab)
    # Additional processing can be added here
    prediction = model.predict(x,verbose=0)
    index = numpy. argmax (prediction)
    result = int_to_char [index]
    seq_in = [int_to_char[value] for value in pattern]
    sys. stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]


In [13]:
# prompt: import numpy as np
# # Generate a random starting index
# start = np.random.randint(0, len(dataX))  # Random index
# pattern = dataX[start]
# # Print the seed
# print("Seed:")
# print("'{}'".format(''.join([int_to_char[value] for value in pattern])))
# # Generate characters
# for i in range(1000):
#     # Reshape the input pattern for the model
#     x = np.reshape(pattern, (1, len(pattern), 1))
#     # Normalize the values
#     x = x / float(n_vocab)
#     # Additional processing can be added here
#     prediction = model.predict(x,verbose=0)
#     index = numpy. argmax (prediction)
#     result = int_to_char [index]
#     seq_in = [int_to_char[value] for value in pattern]
#     sys. stdout.write(result)
#     pattern.append(index)
#     pattern = pattern[1:len(pattern)]
#  check thi socede why it is not abley o generate further based on seed

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import sys

# ... (Your existing code for data loading, preprocessing, and model training)

# ... (Your model loading code)

# With this line: (Corrected int_to_char creation)
int_to_char = dict((i, c) for i, c in enumerate(chars))

# Generate a random starting index
start = np.random.randint(0, len(dataX))  # Random index
pattern = list(dataX[start]) # Convert to list for mutability


# Print the seed
print("Seed:")
print("'{}'".format(''.join([int_to_char[value] for value in pattern])))

# Generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction) # Use numpy.argmax
    result = int_to_char[index]
    sys.stdout.write(result) # Correct stdout
    pattern.append(index)
    pattern = pattern[1:len(pattern)] #Correct slicing

Seed:
' herself, as she wandered about in the wood, “is to grow to my right size again; and the second thin'
g if tome  int hemt de the hase aade ger hene mean her  and feongdg atite aates,as she want herting oo hem  and toink da anvel  and ooonedd tot anr thrt ditn lo anl she wist de the mimt  and she qert dodng puupes on the same, and she qeet quite ao oo eem coosd all ser want wo to she woate  t he the querl sfther bo soee as the diol, aad  alite celi the ras ceard iou ii ger her, and then her lo tee hooee aader, af  am chollos tasy hac no aogoee an har fand “has she rast oo the iing  and ae rery doonn oos rooe oa die eiteh aarrne the seete rab eep tee tootee and she rert wonn io to vos mame mo tar mooe aa in eer hooeng an io,reit  aat in thi kerr oo thi kent  nn t in the that had aever and the sere whr seat no tare oo ne rhy leke  and ae nar quite aerere teet hn  an th  ou the could aov ree hid athsnne to tes see wabbit  aut whe  an cellte the kaad “undd  to the sooen sard thet nn 

In [ ]:
# import numpy as np
# import sys
#other way implementing it using variation in temperarie

# # Sampling function with temperature scaling
# def sample_with_temperature(predictions, temperature=1.0):
#     predictions = np.asarray(predictions).astype("float64")
#     predictions = np.log(predictions + 1e-8) / temperature
#     exp_predictions = np.exp(predictions)
#     predictions = exp_predictions / np.sum(exp_predictions)
#     return np.random.choice(len(predictions), p=predictions)

# # Generate a random starting index
# start = np.random.randint(0, len(dataX))
# pattern = dataX[start]

# # Print the seed
# seed_text = ''.join([int_to_char[value] for value in pattern])
# print("Seed Text:")
# print(f"'{seed_text}'")

# # Generate characters
# print("\nGenerated Text:")
# generated_text = ""
# for i in range(1000):  # Generate 1000 characters
#     x = np.reshape(pattern, (1, len(pattern), 1))
#     x = x / float(n_vocab)

#     # Predict the next character
#     prediction = model.predict(x, verbose=0)
#     index = sample_with_temperature(prediction[0], temperature=0.7)  # Moderate temperature for balance
#     result = int_to_char[index]
#     generated_text += result
#     sys.stdout.write(result)
#     sys.stdout.flush()

#     # Update the pattern
#     pattern.append(index)
#     pattern = pattern[1:]

# print("\n\nPrediction complete.")


Attentive Chatbot